In [1]:
import sys
sys.path.insert(0,'../scr')
from network_prelim import *
from Gephi import create_grad_graph_w_subgraphs_graphml as gml

### Obtain `results` output for small example network (SUN)

Note that the SUN network is very small compaired to the networks FullConn and Strongedges. If you want to run either of these examples, expect 15 to 30 minutes for code to run on a desktop with decent computational power.

In [2]:
network_filename = 'SUN'
network_location = "../networks/" +  network_filename
with open(network_location + ".txt","r") as f:
    network = f.read()
results = get_results_from_string(network, network_location)

pg size 8640
2023-05-15 10:13:08.003188:
MonostableQuery :: initializing
2023-05-15 10:13:08.003509:
MonostableQuery :: select MorseGraphIndex from (select MorseGraphIndex, count(*) as StableCount from (select MorseGraphIndex,Vertex from MorseGraphVertices except select MorseGraphIndex,Source from MorseGraphEdges) group by MorseGraphIndex) where StableCount=1;
2023-05-15 10:13:08.004135:
MonostableQuery :: constructed
4846
cG size 489
True 2
True 2
diagP size 203
True
('../networks/SUN', {'PG size': 8640, 'G size': 4846, 'G edges': 27653, 'cG size': 489, 'cG edges': 2317, 'P size': 238, 'P edges': 697, 'diagP size': 203, 'diagP edges': 578, 'diagP nodes in G': 2406, 'diagP edges in G': 4575, 'path exists': True, 'WCut': 0.08994361107942998, 'eigval': -0.3483, 'Ck cut': {0: 0.008553322540029776, 1: 0.08139028853940021}, 'Ck start/stop': [[], [0, 1], [484, 486, 487], []], 'markov_results': {'B sum': 1.0, 0: 0.0020194551001316563, 1: 0.00045266551822338097, 'leak': 0.6086994113874354, 'rp

#### Breaking down code above into steps and making .graphml file for Gephi (SUN Example)

In [3]:
network_filename = 'SUN'
network_location = "../networks/" +  network_filename
database = Database(network_location+'.db')
with open(network_location+'.txt',"r") as f:
        network = f.read()


pg = ParameterGraph(database.network)
print('pg size',pg.size())
out_edges = get_number_out_edges_from_string(network)
Hb_list, Kni_list = get_Hb_Kni_list(database)
Hb_max = len(Hb_list)-1
Kni_max = len(Kni_list)-1

FP_Poset, FP_Regions = get_FP_Poset(out_edges)

# Compute G (chemical gradient graph) and save.

G = get_grad_graph_strict_bagged(database, network)

# Compute condensation cG of G

strongcc = strongly_connected_components_by_MGI(G, database)
cG, scc = condensation(G, strongcc)
print('cG size', len(cG))

# Make cG a networkx object
N = nx.DiGraph()
for node in cG:
    N.add_node(node)
    for edge in cG[node]:
        N.add_edge(node, edge)

# Add edge weights to cG (now N)
N, scc_edges = add_source_weight_to_cond(G, N, scc, save_count = True)

# Obtain starting and stopping nodes of cG
start_set, stop_set = return_start_stop_set(database, cG, scc, Hb_max, Kni_max, FP_Regions)

# Compute Tensor Product graph (P) and the restricted diagonal Product (diagP)
P = get_product_graph(database, N, scc, FP_Poset, start_set, stop_set)

lenP_nodes = len(P.nodes())
lenP_edges = len(P.edges())

breaks = find_breaks_in_FG_comb(database, network_filename, P, scc, Hb_max, Kni_max, FP_Regions) 
keep = build_diag(Hb_max, Kni_max, breaks)
diagP = remove_unnecessary_nodes_in_P(P, keep, scc)

reachability(diagP, start_set, stop_set)
print('diagP size', len(diagP))
edges_in_G = 0
nodes_in_G = 0
for n in diagP:
    nodes_in_G += len(scc[n])
    for e in diagP[n]:
        edges_in_G += scc_edges[(n,e)]

plot_FG_layer_comb_in_G(diagP, scc, Hb_max, network_filename+'scc_FG_layer_combos')


# Test if any paths exist
path_exists = any_path_exists(diagP, start_set, stop_set)
print('Path Exists = ', path_exists)

if path_exists == True:
    c, eigval, m, C1, C2, Ck_cut = find_best_clustering(diagP, start_set, stop_set, network_filename, 20, nodelist = None, data = 'weight', in_out_degree = 'out', save_file = True, general_cut=True)
else:
    c, eigval, m, C1, C2, Ck_cut = 0, 0, 0, [], [], 0

C1s = list(set(start_set).intersection(C1))
C1t = list(set(stop_set).intersection(C1))
C2s = list(set(start_set).intersection(C2))
C2t = list(set(stop_set).intersection(C2))

mP =  P_with_absorbing_nodes(database, N, diagP, scc, FP_Regions, stop_set)
markov_results = absorbing_Markov_prob(mP, scc, start_set)

results = (network_filename, {'PG size': pg.size(), 'G size': len(G.nodes()), 'G edges': len(G.edges()), 'cG size': len(cG.nodes()), 'cG edges': len(cG.edges()), 'P size' : lenP_nodes, 'P edges': lenP_edges, 'diagP size': len(diagP.nodes()), 'diagP edges': len(diagP.edges()), 'diagP nodes in G': nodes_in_G, 'diagP edges in G': edges_in_G, 'path exists': path_exists, 'WCut': c, 'eigval': eigval, 'Ck cut': Ck_cut, 'Ck start/stop': [C1s, C1t, C2s, C2t], 'markov_results': markov_results})

print(results, flush=True)

pg size 8640
2023-05-15 10:13:10.829195:
MonostableQuery :: initializing
2023-05-15 10:13:10.829508:
MonostableQuery :: select MorseGraphIndex from (select MorseGraphIndex, count(*) as StableCount from (select MorseGraphIndex,Vertex from MorseGraphVertices except select MorseGraphIndex,Source from MorseGraphEdges) group by MorseGraphIndex) where StableCount=1;
2023-05-15 10:13:10.830104:
MonostableQuery :: constructed
4846
cG size 489
True 2
True 2
diagP size 203
Path Exists =  True
('SUN', {'PG size': 8640, 'G size': 4846, 'G edges': 27653, 'cG size': 489, 'cG edges': 2317, 'P size': 238, 'P edges': 697, 'diagP size': 203, 'diagP edges': 578, 'diagP nodes in G': 2406, 'diagP edges in G': 4575, 'path exists': True, 'WCut': 0.08994361107942998, 'eigval': -0.3483, 'Ck cut': {0: 0.008553322540029776, 1: 0.08139028853940021}, 'Ck start/stop': [[], [0, 1], [484, 486, 487], []], 'markov_results': {'B sum': 1.0, 0: 0.0020194551001316563, 1: 0.00045266551822338097, 'leak': 0.6086994113874354, 

In [4]:
gml(database, scc, N, diagP, P, FP_Regions, start_set, stop_set, network_location+'.graphml')